# Imports

In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re

# Carregando Dados

In [22]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE = '2024-02-16 - apartamentos - cilar.csv'
TEMP_FILE = 'local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

blop = bucket.blob(blob_name= FILE).download_as_string()
with open(TEMP_FILE, 'wb') as f:
    f.write(blop)

df = pd.read_csv('local.csv')

In [26]:
df['detalhes'][0].replace("['", "").replace("']", "").replace("', '", ' ').replace('Características do imóvel ', '')

'Área Total 421,62m2 Quartos 3 Suítes 3 Andar 15'

In [24]:
aux = ['Características', 'do', 'imóvel', 'Área', 'Total', '421,62m2', 'Quartos', '3', 'Suítes', '3', 'Andar', '15']

In [27]:
' '.join(aux).replace('Características do imóvel ', '')

'Área Total 421,62m2 Quartos 3 Suítes 3 Andar 15'

# Tratando Dados

## Limpeza

In [17]:
df['detalhes'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else x.replace("['", "").replace("']", "").replace("', '", ' ').replace('Características do imóvel ', ''))


In [ ]:

df['iptu'] = df[['condominio','iptu']].apply(lambda x: 
                                             x['iptu'] if (isinstance(x['condominio'],float)) else 
                                             x['condominio'] if ('IPTU' in x['condominio']) else x['iptu'], axis = 1)

df['condominio'] = df[['condominio','iptu']].apply(lambda x: 
                                             x['condominio'] if (isinstance(x['condominio'],float)) else 
                                             np.nan if ('IPTU' in x['condominio']) else x['condominio'], axis = 1)

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else int(x.replace('Aluguel','').replace('R$','').replace(',00','').replace('.','')))
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x, float) else float(x.replace('Condominio  ','').replace('R$ ', '').replace('.','').replace(',','.')))
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x, float) else float(x.replace('IPTU  ','').replace('R$ ','').replace('.',',').replace(',','.')))

# Criação de Features

In [5]:
def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

In [6]:
## Detalhes do imóvel
df['area'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Área Total'))
df['quartos'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Quartos'))
df['suites'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Suítes'))
df['banheiros'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Banheiros'))
df['andar'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Andar'))
df['vagas_garagem'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Vagas'))

# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-2])
df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-1])

# Atributos do imóvel e condomínio
df['mobiliado'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in x.lower() else 'Não')
df['piscina'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in x.lower() else 'Não')
df['academia'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in x.lower() else 'Não')
df['sacada'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in x.lower() else 'Não')
df['churrasqueira'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in x.lower() else 'Não')


# Save

In [7]:
columns_selected = [
 'titulo',
 'link',
 'endereco',
 'aluguel',
 'condominio',
 'iptu',
 'catacteristicas_imovel',
 'detalhes_condominio',
 'area',
 'quartos',
 'suites',
 'banheiros',
 'andar',
 'vagas_garagem',
 'bairro',
 'cidade',
 'mobiliado',
 'piscina',
 'academia',
 'sacada',
]

In [8]:
df[columns_selected].to_csv('../data/data_trusted/anuncios-cilar-trusted.csv', index = False)

OSError: Cannot save file into a non-existent directory: '../data/data_trusted'